# **Deep Learing Models**

# LSTM Model

In [ ]:

import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE





# Load the dataset
df = pd.read_csv('/content/cleaned_dataset (1).csv')

# Convert string representation of list to actual list
df['tweet_tokens'] = df['tweet_tokens'].apply(ast.literal_eval)

# Define feature and target variables
X = df[['tweet_tokens']]
y = df['class']


# Define a function to join tokens into a single string
X['tweet_tokens'] = X['tweet_tokens'].apply(lambda x: ' '.join(x))

# Use TfidfVectorizer for the 'tweet_tokens' column
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X['tweet_tokens'])

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Handle class imbalance by converting labels to one-hot encoding
y_categorical = to_categorical(y_encoded)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_categorical, test_size=0.2, random_state=42)

<ipython-input-13-7dbc1a73ba5e>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['tweet_tokens'] = X['tweet_tokens'].apply(lambda x: ' '.join(x))


In [ ]:
# Define the LSTM model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_tfidf.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.001)

# Sort the indices of the sparse matrix
X_tfidf.sort_indices()

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    verbose=1)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy:.3f}')

Epoch 1/10
620/620 [==============================] - 15s 21ms/step - loss: 0.4567 - accuracy: 0.8418 - val_loss: 0.3288 - val_accuracy: 0.8808
Epoch 2/10
620/620 [==============================] - 14s 22ms/step - loss: 0.2135 - accuracy: 0.9239 - val_loss: 0.3402 - val_accuracy: 0.8790
Epoch 3/10
620/620 [==============================] - 14s 22ms/step - loss: 0.1264 - accuracy: 0.9561 - val_loss: 0.4009 - val_accuracy: 0.8679
Epoch 4/10
620/620 [==============================] - 12s 20ms/step - loss: 0.0818 - accuracy: 0.9716 - val_loss: 0.4531 - val_accuracy: 0.8634
Epoch 5/10
620/620 [==============================] - 12s 19ms/step - loss: 0.0571 - accuracy: 0.9795 - val_loss: 0.5322 - val_accuracy: 0.8560
Epoch 6/10
155/155 [==============================] - 0s 2ms/step - loss: 0.5910 - accuracy: 0.8608
Validation accuracy: 0.861
